In [1]:
import pandas as pd
import numpy as np
import os,os.path
from joblib import Parallel, delayed
import warnings
from dateutil.relativedelta import relativedelta
import collections.abc
import datetime

In [2]:
data_path = f"./hfq_multi.parquet"

In [3]:
hfq_data = pd.read_parquet(data_path)

In [4]:
hfq_data.head()

,000001.XSHE,000002.XSHE,000004.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000011.XSHE,000012.XSHE,000014.XSHE,...,688376.XSHG,301165.XSHE,301365.XSHE,301377.XSHE,688480.XSHG,301290.XSHE,301311.XSHE,301391.XSHE,688489.XSHG,001256.XSHE
2015-01-05,77.371,114.473,7.446,41.295,8.035,6.073,NaN,3.458,19.952,9.029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,77.371,114.473,7.446,41.295,8.035,6.073,NaN,3.458,19.952,9.029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,77.371,114.473,7.446,41.295,8.035,6.073,6.084,3.458,19.952,9.029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,77.371,114.473,7.446,41.295,8.035,6.073,6.084,3.458,19.952,9.029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-09,77.371,114.473,7.446,41.295,8.035,6.073,6.084,3.458,19.952,9.029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
hfq_multi_daily = hfq_data

In [6]:
hfq_multi = hfq_multi_daily.reindex(hfq_multi_daily.index.union([hfq_multi_daily.index[-1] + relativedelta(days=+1)])).resample(f'{period}T',label='right').ffill()

NameError: name 'period' is not defined

In [ ]:
def resample_hfq_multi(hfq_multi_daily,dates,period):
    hfq_multi = hfq_multi_daily.reindex(hfq_multi_daily.index.union([hfq_multi_daily.index[-1] + relativedelta(days=+1)])).resample(f'{period}T',label='right').ffill()
    hfq_multi = hfq_multi.loc[hfq_multi.index.normalize().isin(dates)]
    hfq_multi = pd.concat([hfq_multi.between_time('09:30','11:30',inclusive='right'),hfq_multi.between_time('13:00','15:00',inclusive='right')],axis=0).sort_index()
    hfq_multi.index.name='time'
    return hfq_multi